In [9]:
from datasets.Dataset_RING import Dataset_RING
import matplotlib.pyplot as plt
import seaborn as sns
import random
import torch
import torch.nn as nn
import torch.nn.parallel
import torch.backends.cudnn as cudnn
import torch.optim as optim
import torch.utils.data
from models.Generator_B1 import Generator_B1  
from models.Discriminator_B1 import Discriminator_B1
from torch.utils.tensorboard import SummaryWriter
from PIL import Image
from utils.performance_measures import *

In [10]:
manualSeed = 99
random.seed(manualSeed)
torch.manual_seed(manualSeed)

writer = SummaryWriter("logs3")

In [11]:
device = torch.device("cuda:0" if (torch.cuda.is_available() and ngpu > 0) else "cpu")
batch_size = 100
num_workers = 2
d_z=100
d_h=128
ngpu=0
beta1=0.5
num_epochs=45
test_sample_size=10000

In [12]:
dataset = Dataset_RING()
dataloader = torch.utils.data.DataLoader(dataset, batch_size=batch_size, shuffle=True, num_workers=num_workers)

_data = dataset[:, :]
X = _data[:,0]
Y = _data[:,1]

plot_scatter = sns.scatterplot(x=X, y=Y)
fig_scatter = plot_scatter.get_figure()
writer.add_figure('Scatter_Plot-Dataset', fig_scatter)

def weights_init(m):
    classname = m.__class__.__name__
    if classname.find('Conv') != -1:
        nn.init.normal_(m.weight.data, 0.0, 0.02)
    elif classname.find('BatchNorm') != -1:
        nn.init.normal_(m.weight.data, 1.0, 0.02)
        nn.init.constant_(m.bias.data, 0)



In [13]:
gen = Generator_B1(d_z=d_z, d_h=d_h)

if (device.type == 'cuda') and (ngpu > 1):
    gen = nn.DataParallel(gen, list(range(ngpu)))

gen.apply(weights_init)

print(gen)

disc = Discriminator_B1(d_h=d_h)

if (device.type == 'cuda') and (ngpu > 1):
    disc = nn.DataParallel(disc, list(range(ngpu)))

disc.apply(weights_init)

print(disc)

Generator_B1(
  (main): Sequential(
    (0): Linear(in_features=100, out_features=128, bias=True)
    (1): ReLU()
    (2): Linear(in_features=128, out_features=128, bias=True)
    (3): ReLU()
    (4): Linear(in_features=128, out_features=2, bias=True)
  )
)
Discriminator_B1(
  (main): Sequential(
    (0): Linear(in_features=2, out_features=128, bias=True)
    (1): ReLU()
    (2): Linear(in_features=128, out_features=128, bias=True)
    (3): ReLU()
    (4): Linear(in_features=128, out_features=1, bias=True)
    (5): Sigmoid()
  )
)


In [14]:
"""
Loss functions and optimisers
"""

# Initialize BCELoss function
criterion = nn.BCELoss()

# Create batch of latent vectors that we will use to visualize
#  the progression of the generator
fixed_noise = torch.randn(test_sample_size, d_z,  device=device)

# Establish convention for real and fake labels during training
real_label = 1
fake_label = 0

lrg = 1e-3
lrd = 2e-3

# Setup Adam optimizers for both G and D
optimizerD = optim.Adam(disc.parameters(), lr=lrd, betas=(beta1, 0.999))
optimizerG = optim.Adam(gen.parameters(), lr=lrg, betas=(beta1, 0.999))

In [16]:
adversary_finding_dataset = Dataset_RING(num_samples=50000)
adversary_finding_dataloader = torch.utils.data.DataLoader(dataset, batch_size=batch_size, shuffle=True, num_workers=num_workers)

test_dataset = Dataset_RING(num_samples=10000)
test_dataloader = torch.utils.data.DataLoader(dataset, batch_size=10000, shuffle=True, num_workers=num_workers)

get_duality_gap(Generator_B1, Discriminator_B1, gen, disc, adversary_finding_dataloader, test_dataloader, criterion, device, 1e-3, 1e-3, writer, num_epochs=1)

DG-Discriminator----Epoch-->0; Iters-->0; Error-->1.3877887725830078
DG-Discriminator----Epoch-->0; Iters-->500; Error-->0.0005193169927224517
DG-Generator----Epoch-->0; Iters-->0; Error-->0.6691757440567017


RuntimeError: Trying to backward through the graph a second time, but the buffers have already been freed. Specify retain_graph=True when calling backward the first time.

In [ ]:
"""
The training 
"""

iters = 0

for epoch in range(num_epochs):
    #for each batch in the dataloader
    for i, data in enumerate(dataloader, 0):
        #######
        # (1) Update D network: maximise log(D(x)) + log(1-D(G(z)))
        #######
        
        ##train with real batch
        disc.zero_grad()
        real_batch = data.float().to(device)

        b_size = real_batch.size(0)
        label = torch.full((b_size,), real_label, device=device)
        # Forward pass real batch through D
        output = disc(real_batch).view(-1)
        # Calculate loss on all-real batch

        errD_real = criterion(output, label)
        # Calculate gradients for D in backward pass
        errD_real.backward()
        D_x = output.mean().item()
        
        ## Train with all-fake batch
        # Generate batch of latent vectors
        noise = torch.randn(b_size, d_z,  device=device)
        # Generate fake image batch with G
        fake = gen(noise)
        label.fill_(fake_label)
        # Classify all fake batch with D
        output = disc(fake.detach()).view(-1)
        # Calculate D's loss on the all-fake batch
        errD_fake = criterion(output, label)
        # Calculate the gradients for this batch
        errD_fake.backward()
        D_G_z1 = output.mean().item()
        # Add the gradients from the all-real and all-fake batches
        errD = errD_real + errD_fake
        # Update D
        optimizerD.step()

        ############################
        # (2) Update G network: maximize log(D(G(z)))
        ###########################
        gen.zero_grad()
        label.fill_(real_label)  # fake labels are real for generator cost
        # Since we just updated D, perform another forward pass of all-fake batch through D
        output = disc(fake).view(-1)
        # Calculate G's loss based on this output
        errG = criterion(output, label)
        # Calculate gradients for G
        errG.backward()
        D_G_z2 = output.mean().item()
        # Update G
        optimizerG.step()


        if (iters%500==0) or ((epoch == num_epochs-1) and (i==len(dataloader)-1)):
            print('Epoch-->{}; Iters-->{}; D_Loss-->{}; G_Loss-->{}'.format(epoch, iters, errD.data.item(), errG.data.item()))
            with torch.no_grad():
                fake = gen(fixed_noise).detach().cpu()
                X = fake[:,0].numpy()
                Y = fake[:,1].numpy()

                plot_kde = sns.kdeplot(data=X, data2=Y, cmap='Reds', shade=True, shade_lowest=False)
                fig_kde = plot_kde.get_figure()
                writer.add_figure('KDE_Plot-Output_on_Fixed_Noise', fig_kde, global_step=iters)

                plot_scatter = sns.scatterplot(x=X, y=Y)
                fig_scatter = plot_scatter.get_figure()
                writer.add_figure('Scatter_Plot-Output_on_Fixed_Noise', fig_scatter, global_step=iters)

        
        #write the losses to tensorboard
        writer.add_scalar('Discriminator-Error', errD.data.item(), global_step=iters)
        writer.add_scalar('Generator-Error', errG.data.item(),  global_step=iters)

        
        iters+=1


In [ ]:
writer.close()
